This notebook is an example for the dataset which annotated as index png image like pascal voc2012.

If the data are annotated as RGB image, modify data_type when you use DataGenereator.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

Set path to this package's src/

In [ ]:
deeplabv3plus_srcdir="./src"
sys.path.append(deeplabv3plus_srcdir)

If you want, write about gpu setting here.

In [ ]:
gpu_options = tf.compat.v1.GPUOptions(visible_device_list="1", allow_growth=True)
config = tf.compat.v1.ConfigProto(gpu_options = gpu_options)
tf.compat.v1.enable_eager_execution(config=config)

In [ ]:
#from image_utils import make_pascal_voc_label_csv
#make_pascal_voc_label_csv()

import from .src/

In [ ]:
from model import deeplab_v3plus_transfer_os16
#from image_utils import make_x_from_image_paths,make_y_from_image_paths,convert_y_to_image_array
from data_gen import DataGenerator
from metrics import IoU
from label import Label

In [ ]:
#import importlib
#importlib.reload(sys.modules['image_utils'])

Learned model and loss curve are exported to out_dir.

In [ ]:
out_dir = "TEST"
os.makedirs(out_dir, exist_ok=True)

Make train_x_paths and train_y_paths as list.

The order of train_x_paths and train_y_paths mast be correspond.

In [ ]:
seg_img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/SegmentationClass"
img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/JPEGImages"
train_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt"
valid_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt"

with open(train_set_path) as f:
    train_img_names = f.read().split("\n")[:-1]
with open(valid_set_path) as f:
    valid_img_names = f.read().split("\n")[:-1]

train_x_paths = np.array([os.path.join(img_dir,train_img_names[i]) + ".jpg" for i in range(len(train_img_names))])
train_y_paths = np.array([os.path.join(seg_img_dir,train_img_names[i]) + ".png" for i in range(len(train_img_names))])

valid_x_paths = np.array([os.path.join(img_dir,valid_img_names[i]) + ".jpg" for i in range(len(valid_img_names))])
valid_y_paths = np.array([os.path.join(seg_img_dir,valid_img_names[i]) + ".png" for i in range(len(valid_img_names))])

Set label_file_path, and image_size. Every image is resize to image_size.

In [ ]:
label_file_path = "./pascal_voc_label.csv"
label = Label(label_file_path)
image_size = (512,512)

set batch_size and n_epochs

In [ ]:
batch_size=16
n_epochs=300

You can choose model from [here](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

If you want to do transfer learning, preprocess must be correspond to the encoder.

layer_name_to_decorder means the layer name which correspond to "Low-Level Features" arrow in Fig.2 of the [paper](https://arxiv.org/pdf/1802.02611.pdf).

encoder_end_layer_name means the layer name input to ASPP block.

deeplab_v3plus_transfer_os16 makes model.
- If you don't want to freeze encoder, write freeze_encoder=False in the function.
- Default activation function of the last layer is softmax. If you want to use sigmoid, write output_activation='sigmoid' in the function.

In [ ]:
encoder = keras.applications.Xception(input_shape=(512,512,3), weights="imagenet", include_top=False)
preprocess = keras.applications.xception.preprocess_input
layer_name_to_decoder = "block3_sepconv2_bn"
encoder_end_layer_name = "block13_sepconv2_bn"
model = deeplab_v3plus_transfer_os16(label.n_labels, encoder, layer_name_to_decoder, encoder_end_layer_name)

Make data generator like this. 

If your dataset is annotated as RGB image, modify data_type="image".

In [ ]:
train_data_gen = DataGenerator(train_x_paths, train_y_paths, image_size, label, batch_size, preprocess, augmentation=True, shuffle=True, data_type="index_png")
valid_data_gen = DataGenerator(valid_x_paths, valid_y_paths, image_size, label, batch_size, preprocess, augmentation=False, shuffle=False, data_type="index_png")

In [ ]:
#model.summary(line_length=150)

Compile the model. You can change it if necessary.

In [ ]:
loss_function = tf.keras.losses.categorical_crossentropy
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt, loss=loss_function, metrics=[IoU])

You can also use callbacks.

In [ ]:
filepath = os.path.join(out_dir,'{epoch:06d}.h5')
cp_cb = keras.callbacks.ModelCheckpoint(filepath, 
                                        monitor='val_IoU', 
                                        verbose=1, 
                                        save_best_only=True, 
                                        save_weights_only=False, 
                                        mode='max')

You can modify freely.

In [ ]:
#hist = model.fit_generator(data_gen, validation_data=(valid_x, valid_y), epochs=par.n_epochs, steps_per_epoch=par.n_batch, callbacks=[cp_cb])
hist = model.fit_generator(train_data_gen,
                           epochs=n_epochs,
                           steps_per_epoch=len(train_data_gen),
                           validation_data=valid_data_gen,
                           validation_steps=len(valid_data_gen),
                           #shuffle = False,
                           workers=8,
                           use_multiprocessing=True,
                           callbacks=[cp_cb])
#hist = model.fit_generator(data_gen, epochs=par.n_epochs, steps_per_epoch=par.n_batch, workers=8, use_multiprocessing=True)

Plot learning curve.

In [ ]:
plt.figure(figsize=(30,10))

plt.subplot(1,3,1)
plt.plot(hist.history["loss"], label="loss")
plt.plot(hist.history["val_loss"], label="val_loss")
plt.legend()

plt.subplot(1,3,2)
plt.plot(hist.history["IoU"], label="IoU")
plt.plot(hist.history["val_IoU"], label="val_IoU")
plt.legend()
plt.savefig(os.path.join(out_dir,'losscurve.png'))

save last epoch model, loss, and metrics,

In [ ]:
model.save(os.path.join(out_dir,'final_epoch.h5'))
for key in sorted(hist.history.keys()):
    np.savetxt(os.path.join(out_dir,key+'.txt'),np.array(hist.history[key]))
